In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
trainfile = '../data/processed/train_1.0.csv'
testfile = '../data/processed/test_1.0.csv'

In [3]:
df_train = pd.read_csv(trainfile)
df_test = pd.read_csv(testfile)

In [4]:
df_train.drop('Unnamed: 0', axis=1,inplace=True)

In [5]:
df_train.sort_values(['class_1','Name']).head()

,PassengerId,Survived,class_1,class_2,Name,Female,FamilySize,inAge_rand_Infant,inAge_rand_Child,inAge_rand_Teenager,...,inAge_rand_Adult,inAge_rand_Senior,inAge_mean_Infant,inAge_mean_Child,inAge_mean_Teenager,inAge_mean_Young Adult,inAge_mean_Adult,inAge_mean_Senior,from_C,Eins
845,846,0,0,0,"Abbing, Mr. Anthony",0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,1.0
746,747,0,0,0,"Abbott, Mr. Rossmore Edward",0,3,0,0,1,...,0,0,0,0,1,0,0,0,0,1.0
279,280,1,0,0,"Abbott, Mrs. Stanton (Rosa Hunt)",1,3,0,0,0,...,0,0,0,0,0,1,0,0,0,1.0
308,309,0,0,1,"Abelson, Mr. Samuel",0,2,0,0,0,...,0,0,0,0,0,1,0,0,1,1.0
874,875,1,0,1,"Abelson, Mrs. Samuel (Hannah Wizosky)",1,2,0,0,0,...,0,0,0,0,0,1,0,0,1,1.0


In [20]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import statsmodels.api as sm

/Users/CodyGette/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [16]:
X_train = df_train[['class_1','class_2','Female',"FamilySize",
                          'inAge_rand_Infant','inAge_rand_Child', 'inAge_rand_Young Adult',
                            'from_C','Eins']]
Y_train = df_train["Survived"]
X_test  = df_test[['class_1','class_2','Female',"FamilySize",
                          'inAge_rand_Infant','inAge_rand_Child', 'inAge_rand_Young Adult',
                            'from_C','Eins']].copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 9), (891,), (891, 9))

In [21]:
logit = sm.Logit(Y_train, X_train)
result_rand = logit.fit()
result_rand.summary()

Optimization terminated successfully.
         Current function value: 0.428299
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      882
Method:                           MLE   Df Model:                            8
Date:                Sun, 18 Mar 2018   Pseudo R-squ.:                  0.3568
Time:                        16:30:15   Log-Likelihood:                -381.61
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 1.817e-86
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
class_1                    2.1109      0.238      8.862      0.000       1.644       2.578
class_2                    1.1141      0.232      4.809      0.000       0.660       1.568
Female                     2.9454      0.208     14.175      0.000       2.538       3.353
FamilySize                -0.3249      0.077     -4.195      0.000      -0.477      -0.173
inAge_rand_Infant          3.0717      0.479      6.414      0.000       2.133       4.010
inAge_rand_Child           1.2898      0.536      2.405      0.016       0.238       2.341
inAge_rand_Young Adult     0.4354      0.199      2.189      0.029       0.046       0.825
from_C                     0.4934      0.234      2.112      0.035       0.036       0.951
Eins                      -2.3568      0.249     -9.466      0.000      -2.845      -1.869
==========================================================================================
"""

In [22]:
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train,Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

82.040000000000006

In [23]:
pd.Series(logreg.coef_[0])

0    2.110613
1    1.114238
2    2.945362
3   -0.324799
4    3.071351
5    1.287919
6    0.435142
7    0.493600
8   -1.178348
dtype: float64

In [24]:
coef = pd.DataFrame(['class_1','class_2','Female',"FamilySize",
                          'inAge_rand_Infant','inAge_rand_Child', 'inAge_rand_Young Adult',
                            'from_C'])
coef.columns = ['Feature']
coef["Correlation"] = pd.Series(logreg.coef_[0])

coef.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
4,inAge_rand_Infant,3.071351
2,Female,2.945362
0,class_1,2.110613
5,inAge_rand_Child,1.287919
1,class_2,1.114238
7,from_C,0.493600
6,inAge_rand_Young Adult,0.435142
3,FamilySize,-0.324799


In [25]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

82.489999999999995

In [26]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

83.609999999999999

In [27]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

84.849999999999994

In [28]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

84.849999999999994

In [29]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,84.85
4,Decision Tree,84.85
1,KNN,83.61
0,Support Vector Machines,82.49
2,Logistic Regression,82.04


In [ ]:
submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('../data/processed/submission.csv', index=False)